In [ ]:
!pip install albumentations==0.4.6
!pip install tensorboard>=2.4.1

In [17]:
%cd '/content/drive/MyDrive/Colab Notebooks/Plant_Pathology'
from typing import List, Dict

import random
import os

import numpy as np
import pandas as pd
import PIL

import albumentations as A
from albumentations.pytorch import ToTensorV2

import torchvision
import torch.onnx
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms as T

import skimage.io as io
from tqdm.notebook import tqdm
import torch
from insectAug import InsectAugmentation


/content/drive/MyDrive/Colab Notebooks/Plant_Pathology


In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.rc('font', size=15)
plt.rc('axes', titlesize=18)  
plt.rc('xtick', labelsize=10)  
plt.rc('ytick', labelsize=10)

In [5]:
class Config: 
    """
    """
    DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
    INPUT_PATH = '/content/drive/MyDrive/Colab Notebooks/Plant_Pathology/data'
    OUTPUT_PATH = './'
    N_EPOCH = 30
    BATCH_SIZE = 16
    TEST_SIZE = 0.2
    RANDOM_STATE = 42
    SAMPLE_FRAC = 1.0
    IMG_SIZE = 224
    LEARNING_RATE = 0.0042319 #for resnet50
    TRAIN_DATA_FILE = os.path.join(INPUT_PATH, 'train.csv')
    MODEL_ONNX_FILE = os.path.join(OUTPUT_PATH, f'plant2021_{DEVICE}.onnx')
    INPUT_MODEL_FILE = os.path.join(INPUT_PATH, f'plant2021_{DEVICE}.pth') 
    OUTPUT_MODEL_FILE = os.path.join(OUTPUT_PATH, f'plant2021_{DEVICE}.pth')
    CLASS_THRESHOLD = 0.4
    CLASSES = [
        'rust', 
        'complex', 
        'healthy', 
        'powdery_mildew', 
        'scab', 
        'frog_eye_leaf_spot'
    ]
    N_CLASSES = len(CLASSES)
    
    folders = dict({
        'data': INPUT_PATH,
        'train': os.path.join(INPUT_PATH, 'train_images'),
        'val': os.path.join(INPUT_PATH, 'train_images'),
        'test':  os.path.join(INPUT_PATH, 'test_images')
    })
    
    @staticmethod
    def set_seed():
        torch.manual_seed(Config.RANDOM_STATE)
        random.seed(Config.RANDOM_STATE)
        np.random.seed(Config.RANDOM_STATE)
        
Config.set_seed() 

In [6]:
def to_numpy(tensor):
    """Auxiliary function to convert tensors into numpy arrays
    """
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

In [7]:
def read_image_labels():
    """
    """
    df = pd.read_csv(Config.TRAIN_DATA_FILE).set_index('image')
    return df

In [8]:
img_labels = read_image_labels().sample(
    frac=Config.SAMPLE_FRAC, 
    random_state=Config.RANDOM_STATE
)

In [10]:
def get_single_labels(unique_labels) -> List[str]:
    """Splitting multi-labels and returning a list of classes"""
    single_labels = []
    
    for label in unique_labels:
        single_labels += label.split()
        
    single_labels = set(single_labels)
    return list(single_labels)

In [11]:
def get_one_hot_encoded_labels(dataset_df) -> pd.DataFrame:
    """
    """
    df = dataset_df.copy()
    
    unique_labels = df.labels.unique()
    column_names = get_single_labels(unique_labels)
    
    df[column_names] = 0        
    
    # one-hot-encoding
    for label in unique_labels:                
        label_indices = df[df['labels'] == label].index
        splited_labels = label.split()
        df.loc[label_indices, splited_labels] = 1
    
    return df

In [12]:
one_hot_encoded_labels = get_one_hot_encoded_labels(img_labels)
one_hot_encoded_labels.head()

,labels,complex,healthy,scab,rust,powdery_mildew,frog_eye_leaf_spot
image,,,,,,,
ae7a52cc3cc3241f.jpg,healthy,0,1,0,0,0,0
b2a08e9bf8f061a7.jpg,scab frog_eye_leaf_spot,0,0,1,0,0,1
806878cdcb8df99a.jpg,scab frog_eye_leaf_spot complex,1,0,1,0,0,1
f5139361351a6b99.jpg,powdery_mildew complex,1,0,0,0,1,0
a39bd3c311438f3c.jpg,healthy,0,1,0,0,0,0


In [13]:
def get_image(image_id, kind='train'):
    """Loads an image from file
    """
    fname = os.path.join(Config.folders[kind], image_id)
    return PIL.Image.open(fname)

In [14]:
def visualize_images(image_ids, labels, nrows=1, ncols=4, kind='train', image_transform=None):
    """
    """
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(20, 8))
    for image_id, label, ax in zip(image_ids, labels, axes.flatten()):
        
        fname = os.path.join(Config.folders[kind], image_id)
        image = np.array(PIL.Image.open(fname))
        
        if image_transform:
            image = transform = A.Compose(
                [t for t in image_transform.transforms if not isinstance(t, (
                    A.Normalize, 
                    ToTensorV2
                ))])(image=image)['image']
        
        io.imshow(image, ax=ax)
        
        ax.set_title(f"Class: {label}", fontsize=12)
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
        
        del image
        
    plt.show()

In [15]:
# visualize_images(img_labels.index, img_labels.labels, nrows=2, ncols=4)

In [18]:
train_transform = A.Compose([
A.OneOf([
       A.Rotate( 
            limit=(-68, 178), 
            interpolation=1, 
            border_mode=1, 
            mask_value=None),
        A.ShiftScaleRotate(
            shift_limit=0.2, 
            scale_limit=0.2, 
            rotate_limit=90)]),
A.OneOf([
       InsectAugmentation(insects=5),
       A.CoarseDropout(
            max_holes=5, 
            max_height=7, 
            max_width=7, 
            min_holes=3, 
            min_height=5, 
            min_width=5),
        A.RandomShadow(
            num_shadows_lower=1, 
            num_shadows_upper=3, 
            shadow_dimension=5)
        ]),
A.OneOf([
        A.RandomFog(
            fog_coef_lower=0.2, 
            fog_coef_upper=0.5, 
            alpha_coef=0.3),
        A.CLAHE(),
        A.RandomBrightnessContrast(),
        A.RGBShift(
            r_shift_limit=20, 
            g_shift_limit=20, 
            b_shift_limit=20),
        A.GaussNoise(
            var_limit=(100, 170))]),
        A.Normalize(
            mean=(0.485, 0.456, 0.406), 
            std=(0.229, 0.224, 0.225)),
        ToTensorV2()])


val_transform = A.Compose([
    A.Resize(
        height=Config.IMG_SIZE,
        width=Config.IMG_SIZE,
    ),
    A.Normalize(
        mean=(0.485, 0.456, 0.406), 
        std=(0.229, 0.224, 0.225)
    ),
    ToTensorV2(),
])

In [19]:
from scipy.stats import bernoulli
from torch.utils.data import Dataset

class PlantDataset(Dataset):
    """
    """
    def __init__(self, 
                 image_ids, 
                 targets,
                 transform=None, 
                 target_transform=None, 
                 kind='train'):
        self.image_ids = image_ids
        self.targets = targets
        self.transform = transform
        self.target_transform = target_transform
        self.kind = kind
    
    def __len__(self):
        return len(self.image_ids)
    
    def __getitem__(self, idx):
        # load and transform image
        img = np.array(get_image(self.image_ids.iloc[idx], kind=self.kind))
        
        if self.transform:
            img = self.transform(image=img)['image']
        
        # get image target 
        target = self.targets[idx]
        if self.target_transform:
            target = self.target_transform(target)
        
        return img, target

In [20]:
from sklearn.model_selection import train_test_split

X_train, X_vaild, y_train, y_vaild = train_test_split(
    pd.Series(img_labels.index), 
    np.array(one_hot_encoded_labels[Config.CLASSES]),  
    test_size=Config.TEST_SIZE, 
    random_state=Config.RANDOM_STATE
)

In [21]:
train_set = PlantDataset(X_train, y_train, transform=train_transform, kind='train')
val_set = PlantDataset(X_vaild, y_vaild, transform=val_transform, kind='val')

In [22]:
print(f'Train size: {len(train_set)}')
print(f'Validation size: {len(val_set)}')

Train size: 14905
Validation size: 3727


In [23]:
from torch.utils.data import DataLoader
from torch.nn import BatchNorm2d

train_loader = DataLoader(train_set, batch_size=Config.BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(val_set, batch_size=Config.BATCH_SIZE, shuffle=True)

In [24]:
def load_model(model, load_path=Config.INPUT_MODEL_FILE):
    model.load_state_dict(torch.load(load_path))
    model.eval()
    
def save_weights(model, save_path=Config.OUTPUT_MODEL_FILE):
    torch.save(model.state_dict(), save_path)

def create_model(pretrained=True):
    model = torchvision.models.resnet50(pretrained=pretrained).to(Config.DEVICE)
    
    for param in model.layer1.parameters():
        param.requires_grad = False
        
    for param in model.layer2.parameters():
        param.requires_grad = False  
        
    for param in model.layer3.parameters():
        param.requires_grad = False 
    
    model.fc = torch.nn.Sequential(
        torch.nn.Linear(
            in_features=model.fc.in_features,
            out_features=Config.N_CLASSES
        ),
        torch.nn.Sigmoid()
    ).to(Config.DEVICE)
    
    return model

In [25]:
model = create_model(pretrained=True).to(Config.DEVICE);

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [26]:
class MetricMonitor:
    def __init__(self):
        self.reset()

    def reset(self):
        self.losses = []
        self.accuracies = []
        self.scores = []
        self.metrics = dict({
            'loss': self.losses,
            'acc': self.accuracies,
            'f1': self.scores
        })

    def update(self, metric_name, value):
        self.metrics[metric_name] += [value]

In [27]:
from sklearn.metrics import f1_score, accuracy_score

def get_metrics(
    y_pred_proba, 
    y_test, 
    threshold=Config.CLASS_THRESHOLD,
    labels=Config.CLASSES) -> None:
    """
    """
    y_pred = np.where(y_pred_proba > threshold, 1, 0)

    y1 = y_pred.round().astype(np.float)
    y2 = y_test.round().astype(np.float)
    
    f1 = f1_score(y1, y2, average='micro')
    acc = accuracy_score(y1, y2, normalize=True)

    return acc, f1

# Hyper parametrs serching

In [ ]:
from torch.utils.tensorboard import SummaryWriter

def hyper_parameter_search(lr_list, bs_list): 
        step = 0
        for batch_size in bs_list:
          for learning_rate in lr_list:
              model = create_model(pretrained=True).to(Config.DEVICE);
              criterion = torch.nn.MultiLabelSoftMarginLoss().cuda()
              optimizer = optim.Adam(model.parameters(), lr=learning_rate)

              train_set = PlantDataset(X_train, y_train, transform=train_transform, kind='train')
              train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
              train_monitor = MetricMonitor()

              writer = SummaryWriter(f'runs/Board/minibatch {batch_size} LR {learning_rate}')
              training_loop(train_loader,model,criterion,optimizer,0,train_monitor)
              writer.add_scalar('Training Loss', train_monitor.losses[0],global_step=step)
              writer.add_scalar('Training F1', train_monitor.scores[0],global_step=step)
              step += 1

# How to create VGG19 model

In [ ]:
def create_model_vgg19(pretrained=True):
    model = torchvision.models.vgg19_bn(pretrained=True).to(Config.DEVICE)
    
    for i in range(len(model.classifier)-1):
         for param in model.classifier[i].parameters():
             param.requires_grad = False
    
    model.classifier[6] = nn.Linear(4096, Config.N_CLASSES)  
    
    return model


# How to create Logistic Regression model 

In [ ]:
class LogisticRegression(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)

    def forward(self, x):
        outputs = self.linear(x)
        return outputs


# training

In [28]:
def training_loop(
    dataloader, 
    model, 
    loss_fn, 
    optimizer, 
    epoch, 
    monitor = MetricMonitor(), 
    is_train=True
) -> None:
    """
    """
    size = len(dataloader.dataset)
    
    loss_val = 0
    accuracy = 0
    f1score = 0
    
    if is_train:
        model.train()
    else:
        model.eval()
    
    stream = tqdm(dataloader)
    for batch, (X, y) in enumerate(stream, start=1):
        X = X.to(Config.DEVICE)
        y = y.to(Config.DEVICE)
        
        # compute prediction and loss
        pred_prob = model(X)
        loss = loss_fn(pred_prob, y)
    
        if is_train:
            # backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        
        loss_val += loss.item()
        acc, f1 = get_metrics(to_numpy(pred_prob), to_numpy(y))
        
        accuracy += acc 
        f1score += f1

        phase = 'Train' if is_train else 'Val'
        stream.set_description(
            f'Epoch {epoch:3d}/{Config.N_EPOCH} - {phase} - Loss: {loss_val/batch:.4f}, ' + 
            f'Acc: {accuracy/batch:.4f}, F1: {f1score/batch:.4f}'
        )

    monitor.update('loss', loss_val/batch)
    monitor.update('acc', accuracy/batch)
    monitor.update('f1', f1score/batch) 

In [29]:
train_monitor = MetricMonitor()
test_monitor = MetricMonitor()

In [30]:
# initialize the loss function
loss_fn = nn.MultiLabelSoftMarginLoss()

optimizer = torch.optim.Adam(
    model.parameters(),
    lr=Config.LEARNING_RATE
)

In [ ]:
%%time

for epoch in range(1, Config.N_EPOCH + 1):
    # training loop
    training_loop(
        train_loader, 
        model, 
        loss_fn, 
        optimizer, 
        epoch, 
        train_monitor,
        is_train=True
    )
    
    # validation loop
    training_loop(
        valid_loader, 
        model, 
        loss_fn, 
        optimizer, 
        epoch, 
        test_monitor,
        is_train=False
    )

In [ ]:
from matplotlib.ticker import MaxNLocator 

def plot_result(
    train_losses, 
    test_losses, 
    train_accuracies, 
    test_accuracies, 
    train_scores,
    test_scores
) -> None:
    
    epochs = range(1, len(train_losses) + 1)
    fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(22, 5))
    
    # plot loss values
    ax[0].plot(epochs, train_losses, label='Training loss', marker ='o')
    ax[0].plot(epochs, test_losses, label='Validation loss', marker ='o')
    ax[0].legend(frameon=False, fontsize=14)
    
    ax[0].get_xaxis().set_major_locator(MaxNLocator(integer=True))
    ax[0].set_title('Loss', fontsize=18)
    ax[0].set_xlabel('Epoch', fontsize=14) 
    ax[0].set_ylabel('Loss', fontsize=14)  
    
    # plot accuracies 
    ax[1].plot(epochs, train_accuracies, label='Training Accuracy', marker ='o')
    ax[1].plot(epochs, test_accuracies, label='Validation accuracy', marker ='o')
    ax[1].legend(frameon=False, fontsize=14)
    
    ax[1].get_xaxis().set_major_locator(MaxNLocator(integer=True))
    ax[1].set_title('Accuracy', fontsize=18)
    ax[1].set_xlabel('Epoch', fontsize=14) 
    ax[1].set_ylabel('Accuracy', fontsize=14)
    
    ax[2].plot(epochs, train_scores, label='Training F1-Score', marker ='o')
    ax[2].plot(epochs, test_scores, label='Validation F1-Score', marker ='o')
    ax[2].legend(frameon=False, fontsize=14)
    
    ax[2].get_xaxis().set_major_locator(MaxNLocator(integer=True))
    ax[2].set_title('F1-Score', fontsize=18)
    ax[2].set_xlabel('Epoch', fontsize=14) 
    ax[2].set_ylabel('F1-Score', fontsize=14) 
        
    plt.show()

In [ ]:


plot_result(
    train_monitor.losses, 
    test_monitor.losses,
    train_monitor.accuracies, 
    test_monitor.accuracies, 
    train_monitor.scores,
    test_monitor.scores
)    



In [ ]:
batch = Config.BATCH_SIZE

y_true = np.empty(shape=(0, 6), dtype=np.int)
y_pred_proba = np.empty(shape=(0, 6), dtype=np.int)

stream = tqdm(valid_loader)
for batch, (X, y) in enumerate(stream, start=1):
    X = X.to(Config.DEVICE)
    y = to_numpy(y.to(Config.DEVICE))
    pred = to_numpy(model(X))
    
    y_true = np.vstack((y_true, y))
    y_pred_proba = np.vstack((y_pred_proba, pred))

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix

def plot_confusion_matrix(
    y_test, 
    y_pred_proba, 
    threshold=Config.CLASS_THRESHOLD, 
    label_names=Config.CLASSES
)-> None:
    """
    """
    y_pred = np.where(y_pred_proba > threshold, 1, 0)
    c_matrices = multilabel_confusion_matrix(y_test, y_pred)
    
    cmap = plt.get_cmap('Blues')
    fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(15, 8))

    for cm, label, ax in zip(c_matrices, label_names, axes.flatten()):
        sns.heatmap(cm, annot=True, fmt='g', ax=ax, cmap=cmap);

        ax.set_xlabel('Predicted labels');
        ax.set_ylabel('True labels'); 
        ax.set_title(f'{label}');

    plt.tight_layout()    
    plt.show()

In [ ]:
plot_confusion_matrix(y_true, y_pred_proba)    

In [ ]:
save_weights(model, '/content/drive/MyDrive/Colab Notebooks/Plant_Pathology/weights/plant2021_cuda.pth')